# Processing
Demonstration of Processing endpoints.

In [ ]:
import utils.DemoClient as client
import jwt
import json

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "185.52.193.87.nip.io"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="demoA"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [ ]:
# ADES URLs
ades_base_url = "http://ades." + platform_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

### ADES API Processes

In [ ]:
ades_access_token = None
app_name = "s-expression-0_0_2"

#### ADES: List Processes

In [ ]:
# API Processes - List Processes
response, process_ids, ades_access_token = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### ADES: Deploy Application

In [ ]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check deploy - list processes again
response, process_ids, ades_access_token = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### ADES: Get Application Details

In [ ]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: Execute Application

In [ ]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

#### ADES: Job Status

In [ ]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

In [ ]:
# API Processes - Job Status (keep polling for completion)
trace = demo.trace_requests; demo.trace_requests = False
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)
demo.trace_requests = trace

In [ ]:
print(response.status_code, response.headers, response.text)

#### ADES: Job Result

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: Undeploy Application

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, process_ids, ades_access_token = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)